<a href="https://colab.research.google.com/github/satoshikawato/gbdraw/blob/main/gbdraw_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gbdraw: a genome diagram generator for microbes

A command-line tool designed for creating detailed diagrams of microbial genomes. gbdraw accepts GenBank/EMBL/DDBJ-format annotated genomes as input and outputs a visual representation of the genomes in SVG/PNG/PDF/EPS/PS formats. This notebook allows you to try selected features of gbdraw on Google Colab without any local installation. For more details, see the [gbdraw GitHub](https://github.com/satoshikawato/gbdraw).

In [ ]:
#@title **1. Install gbdraw from GitHub repository**
%%time
import os

GBDRAW_READY = "GBDRAW_READY"

if not os.path.isfile(GBDRAW_READY):
    print("Cloning and installing gbdraw from GitHub...")
    if not os.path.isdir("gbdraw-main"):
        os.system("git clone https://github.com/satoshikawato/gbdraw.git gbdraw-main")

    os.chdir("gbdraw-main")
    os.system("pip install -q .")
    os.chdir("..")
    os.system(f"touch {GBDRAW_READY}")

print("Successfully installed")

## **2. Run gbdraw**

In [ ]:
#@title  Circular mode
import os, shutil, uuid
import subprocess, time, pathlib
from IPython.display import display, Image, SVG, FileLink
from google.colab import files
import ipywidgets as wd

INPUT_DIR = "gbdraw_input"
os.makedirs(INPUT_DIR, exist_ok=True)

# ---------- helpers ----------
def C_safe(fname: str) -> str:
    base = fname.replace(" ", "_").replace("(", "").replace(")", "")
    dst  = os.path.join(INPUT_DIR, base)
    if os.path.exists(dst):
        stem, ext = os.path.splitext(base)
        base = f"{stem}_{uuid.uuid4().hex[:6]}{ext}"
    return base

def C_refresh(*_):
    files = [""] + sorted(os.listdir(INPUT_DIR))
    C_gb.options = files
    if C_gb.value not in files:
        C_gb.value = ""
    if C_fmt.value not in C_fmt.options:
        C_fmt.value = "png"

# ---------- upload ----------
C_upload = wd.Button(description="📁 Upload files")
def C_uploader(_):
    up = files.upload()
    for raw in up:
        shutil.move(raw, os.path.join(INPUT_DIR, C_safe(raw)))
    C_refresh()
    print("✅ Uploaded:", ", ".join(up.keys()))
C_upload.on_click(C_uploader)

C_refresh_btn = wd.Button(description="🔄 Refresh list")
C_refresh_btn.on_click(C_refresh)

# ---------- widgets ----------
C_gb   = wd.Dropdown(description="GenBank file:", options=[""])
C_trk  = wd.Dropdown(description="Track type:", options=["tuckin","middle","spreadout"], value="tuckin")
C_fmt  = wd.Dropdown(description="Output fmt:", options=["png","svg","pdf","eps","ps"], value="png")
C_legend = wd.Dropdown(description="Legend:", options=["auto","left","right","upper_left","upper_right","lower_left","lower_right"], value="auto")
C_pref = wd.Text(value="", description="Output prefix (optional):")   # ← default blank

C_chk_lbl = wd.Checkbox(description="Show labels")
C_chk_str = wd.Checkbox(description="Separate strands")
# advanced
C_adv_nt, C_adv_win, C_adv_step = (
    wd.Text(value="GC", description="nt (--nt):"),
    wd.IntText(value=1000, description="window:"),
    wd.IntText(value=100,  description="step:")
)
C_adv_feat = wd.Text(value="CDS,tRNA,rRNA,repeat_region", description="features (-k):")

C_adv_box = wd.VBox([
    wd.HTML("<b>Advanced options</b>"),
    wd.HBox([C_adv_nt, C_adv_win, C_adv_step]),
    C_adv_feat
])
C_adv_box.layout.display = "none"
C_toggle_adv = wd.ToggleButton(description="🔧 Advanced")
C_toggle_adv.observe(lambda c: setattr(C_adv_box.layout, "display", None if c["new"] else "none"), names="value")

# ---------- run ----------
C_run = wd.Button(description="🚀 Run gbdraw", button_style="success")

def C_do_run(_):
    C_refresh()
    if not C_gb.value:
        print("❌ Select a GenBank file."); return

    # ---------- build command ----------------------------------
    cmd = ["gbdraw", "circular",
           "-i", os.path.join(INPUT_DIR, C_gb.value),
           "-f", C_fmt.value,
           "--track_type", C_trk.value]

    if C_pref.value.strip():                # user-supplied prefix
        cmd += ["-o", C_pref.value.strip()]
    if C_chk_lbl.value: cmd.append("--show_labels")
    if C_chk_str.value: cmd.append("--separate_strands")
    if C_legend.value != "auto": cmd += ["-l", C_legend.value]
    if C_toggle_adv.value:
        cmd += ["-n", C_adv_nt.value,
                "-w", str(C_adv_win.value),
                "-s", str(C_adv_step.value),
                "-k", C_adv_feat.value]

    print("🛠️", " ".join(cmd))

    # ---------- mtime snapshot BEFORE --------------------------
    before = {f: os.path.getmtime(f) for f in os.listdir(".")}

    # ---------- run & capture ---------------------------------
    res = subprocess.run(cmd, text=True, capture_output=True)
    if res.stdout: print(res.stdout)
    if res.stderr: print(res.stderr)
    if res.returncode != 0:
        print(f"❌ gbdraw exited with code {res.returncode}"); return

    # ---------- AFTER -----------------------------------------
    after = {f: os.path.getmtime(f) for f in os.listdir(".")}

    changed = sorted(f for f,m in after.items()
                     if f not in before or m > before[f] + 1e-6)
    if not changed:
        print("❌ No new or updated output files found."); return

    for f in changed:
        print("📌", f)
        if f.endswith(".svg"):
            display(SVG(f))
        elif f.endswith(".png"):
            display(Image(f))
        else:
            display(FileLink(f))

C_run.on_click(C_do_run)

# ---------- layout ----------
display(wd.VBox([
    wd.HTML("<h2>🧬 gbdraw circular interface</h2>"),
    wd.HBox([C_upload, C_refresh_btn]),
    C_gb, C_trk, C_fmt, C_pref, C_legend,
    wd.HBox([C_chk_lbl, C_chk_str]),
    C_toggle_adv, C_adv_box,
    C_run
]))

In [ ]:
#@title  Linear mode
import os, shutil, uuid
import subprocess, pathlib
from IPython.display import display, Image, SVG, FileLink
from google.colab import files
import ipywidgets as wd

INPUT_DIR, OUT_DIR = "gbdraw_input"
os.makedirs(INPUT_DIR, exist_ok=True)

# ---------- helpers ----------
def L_safe(fname: str) -> str:
    base = fname.replace(" ", "_").replace("(", "").replace(")", "")
    dst  = os.path.join(INPUT_DIR, base)
    if os.path.exists(dst):
        stem, ext = os.path.splitext(base)
        base = f"{stem}_{uuid.uuid4().hex[:6]}{ext}"
    return base

def L_refresh(*_):
    files = [""] + sorted(os.listdir(INPUT_DIR))
    for d in L_SEQ + L_COMP:
        d.options = files
        if d.value not in files:
            d.value = ""
    if L_fmt.value not in L_fmt.options:
        L_fmt.value = "png"

def L_rebuild():
    L_refresh()
    kids = []
    for i, s in enumerate(L_SEQ):
        kids.append(s)
        if i < len(L_SEQ)-1:
            kids.append(L_COMP[i])
    L_box.children = tuple(kids)

# ---------- upload ----------
L_upload = wd.Button(description="📁 Upload files")
def L_uploader(_):
    up = files.upload()
    for raw in up:
        shutil.move(raw, os.path.join(INPUT_DIR, L_safe(raw)))
    L_refresh()
    print("✅ Uploaded:", ", ".join(up.keys()))
L_upload.on_click(L_uploader)

L_refresh_btn = wd.Button(description="🔄 Refresh list")
L_refresh_btn.on_click(L_refresh)

# ---------- sequence / comparison widgets ----------
L_SEQ  = [wd.Dropdown(description="Sequence 1:", options=[""])]
L_COMP = []       # start with zero comparisons
def L_add_pair(_):
    sid = len(L_SEQ)+1
    cid = len(L_COMP)+1
    L_SEQ.append(wd.Dropdown(description=f"Sequence {sid}:",   options=[""]))
    L_COMP.append(wd.Dropdown(description=f"Comparison {cid}:", options=[""]))
    L_rebuild()
L_more = wd.Button(description="➕ Add Seq/Comp pair")
L_more.on_click(L_add_pair)

# ---------- basic / advanced widgets ----------
L_chk_align = wd.Checkbox(description="Align center")
L_chk_sep   = wd.Checkbox(description="Separate strands")
L_chk_gc    = wd.Checkbox(description="Show GC")
L_chk_lbl   = wd.Checkbox(description="Show labels")
L_chk_res   = wd.Checkbox(description="Resolve overlaps")
L_legend    = wd.Dropdown(description="Legend:", options=["auto","left","right"], value="auto")
L_fmt       = wd.Dropdown(description="Output fmt:",
                          options=["png","svg","pdf","eps","ps"],
                          value="png")
L_pref      = wd.Text(value="linear", description="Output prefix:")

L_nt, L_win, L_step = (
    wd.Text(value="GC", description="nt (--nt):"),
    wd.IntText(value=1000, description="window:"),
    wd.IntText(value=100,  description="step:")
)
L_bits  = wd.FloatText(value=50,   description="bitscore:")
L_eval  = wd.FloatText(value=1e-2, description="evalue:")
L_ident = wd.FloatText(value=0,    description="identity:")
L_feats = wd.Text(value="CDS,tRNA,rRNA,repeat_region", description="features (-k):")

L_adv_box = wd.VBox([
    wd.HTML("<b>Advanced options</b>"),
    wd.HBox([L_nt, L_win, L_step]),
    wd.HBox([L_bits, L_eval, L_ident]),
    L_feats
])
L_adv_box.layout.display = "none"
L_toggle_adv = wd.ToggleButton(description="🔧 Advanced")
L_toggle_adv.observe(lambda c: setattr(L_adv_box.layout, "display", None if c["new"] else "none"), names="value")

# ---------- run ----------
L_run = wd.Button(description="🚀 Run gbdraw", button_style="success")

def L_do_run(_):
    seqs = [d.value for d in L_SEQ if d.value]
    if not seqs:
        print("❌ Select at least one Sequence file."); return
    seq_paths = [os.path.join(INPUT_DIR, f) for f in seqs]

    comps   = [d.value for d in L_COMP]
    filled  = [c for c in comps if c]
    if filled and len(filled) != len(L_COMP):
        print("❌ Fill all Comparison slots or leave all empty."); return
    comp_paths = [os.path.join(INPUT_DIR, f) for f in filled]

    # ---------- build command ----------------------------------
    cmd = ["gbdraw", "linear",
           "-i", *seq_paths,
           "-o", L_pref.value.strip() or "linear",
           "-f", L_fmt.value,
           "-n", L_nt.value, "-w", str(L_win.value), "-s", str(L_step.value),
           "--bitscore", str(L_bits.value),
           "--evalue",  str(L_eval.value),
           "--identity", str(L_ident.value),
           "-k", L_feats.value]

    if L_chk_align.value: cmd.append("--align_center")
    if L_chk_sep.value:   cmd.append("--separate_strands")
    if L_chk_gc.value:    cmd.append("--show_gc")
    if L_chk_lbl.value:   cmd.append("--show_labels")
    if L_chk_res.value:   cmd.append("--resolve_overlaps")
    if L_legend.value != "auto": cmd += ["-l", L_legend.value]
    if comp_paths: cmd += ["-b", *comp_paths]

    print("🛠️", " ".join(cmd))

    # ---------- mtime BEFORE -----------------------------------
    before = {f: os.path.getmtime(f) for f in os.listdir(".")}

    # ---------- run & capture ----------------------------------
    res = subprocess.run(cmd, text=True, capture_output=True)
    if res.stdout: print(res.stdout)
    if res.stderr: print(res.stderr)
    if res.returncode != 0:
        print(f"❌ gbdraw exited with code {res.returncode}"); return

    # ---------- AFTER ------------------------------------------
    after = {f: os.path.getmtime(f) for f in os.listdir(".")}

    changed = sorted(f for f,m in after.items()
                     if f not in before or m > before[f] + 1e-6)
    if not changed:
        print("❌ No new or updated output files found."); return

    for f in changed:
        print("📌", f)
        if f.endswith(".svg"):
            display(SVG(f))
        elif f.endswith(".png"):
            display(Image(f))
        else:
            display(FileLink(f))

L_run.on_click(L_do_run)

# ---------- layout ----------
L_box = wd.VBox()
L_rebuild()

display(wd.VBox([
    wd.HTML("<h2>🧬 gbdraw linear interface</h2>"),
    wd.HBox([L_upload, L_refresh_btn]),
    L_box,
    L_more,
    wd.HBox([L_chk_align, L_chk_sep]),
    wd.HBox([L_chk_gc, L_chk_lbl]),
    wd.HBox([L_chk_res, L_legend]),
    L_fmt, L_pref,
    L_toggle_adv, L_adv_box,
    L_run
]))